##### Copyright 2024 Google LLC.

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemini API: Audio Quickstart

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google-gemini/cookbook/blob/main/quickstarts/Audio.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
</table>

This notebook provides an example of how to prompt Gemini 1.5 Pro using an audio file. In this case, you'll use a [sound recording](https://www.jfklibrary.org/asset-viewer/archives/jfkwha-006) of President John F. Kennedy’s 1961 State of the Union address.

In [ ]:
!pip install -q -U google-generativeai

In [ ]:
import google.generativeai as genai

## Configure your API key

To run the following cell, your API key must be stored it in a Colab Secret named `GOOGLE_API_KEY`. If you don't already have an API key, or you're not sure how to create a Colab Secret, see [Authentication](https://github.com/google-gemini/cookbook/blob/main/quickstarts/Authentication.ipynb) for an example.

In [ ]:
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

## Upload an audio file with the File API

To use an audio file in your prompt, you must first upload it using the [File API](https://github.com/google-gemini/cookbook/blob/main/quickstarts/File_API.ipynb).


In [2]:
URL = "https://storage.googleapis.com/generativeai-downloads/data/State_of_the_Union_Address_30_January_1961.mp3"

In [3]:
!wget -q $URL -O sample.mp3

In [4]:
your_file = genai.upload_file(path='sample.mp3')

## Use the file in your prompt

In [5]:
prompt = "Listen carefully to the following audio file. Provide a brief summary."
model = genai.GenerativeModel('models/gemini-1.5-flash')
response = model.generate_content([prompt, your_file])
print(response.text)

President John F. Kennedy delivers his State of the Union address to the United States Congress on January 30th, 1961. He addresses issues concerning the nation's economy, foreign policy, and the state of the executive branch.  The President acknowledges the nation's economic struggles, including high unemployment and a failing agricultural market, but he remains optimistic. He emphasizes the importance of maintaining a strong military, working with allies, and bolstering the United Nations. He suggests expanding a "Food for Peace" program to help alleviate hunger and malnutrition, and he urges collaboration with the Soviet Union on scientific endeavors. The President concludes his speech by acknowledging the challenges ahead and urging Americans to work together to overcome them. 


## Count audio tokens

You can count the number of tokens in your audio file like this.

In [6]:
model.count_tokens([your_file])

total_tokens: 83552

## Learning more

* Learn more about the [File API](https://github.com/google-gemini/cookbook/blob/main/quickstarts/File_API.ipynb) with the quickstart.

* Learn more about prompting with [media files](https://ai.google.dev/tutorials/prompting_with_media) in the docs, including the supported formats and maximum length for audio files.